### RAG Proof of Concept

In [4]:
%pip install -U newsapi-python llama-index

Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
from newsapi import NewsApiClient
import os

news_api_key = os.getenv('NEWS_API_KEY')
newsapi = NewsApiClient(api_key=news_api_key)
get_articles_now = False
articles_path = './saved_articles.json'

if get_articles_now:
    try:
        all_articles = newsapi.get_everything(q='Llama2',
                                              from_param='2023-11-01',
                                              to='2023-11-30',
                                              language='en',
                                              sort_by='publishedAt',
                                              page_size=100)
        with open(articles_path, 'w') as f:
            json.dump(all_articles, f)
    except Exception as e:
        print("An error occurred:", str(e))
else:
    with open(articles_path, 'r') as f:
        all_articles = json.load(f)

print(f'retrieved {len(all_articles["articles"])} articles')

retrieved 100 articles


In [6]:
all_articles['articles'][0]

{'source': {'id': None, 'name': 'Biztoc.com'},
 'author': 'venturebeat.com',
 'title': 'Researchers scanned public repos and found 1,681 exposed Hugging Face API tokens belonging to Meta, Microsoft, Google, and others, many with write permissions',
 'description': 'Further validating how brittle the security of generative AI models and their platforms are, Lasso Security helped Hugging Face dodge a potentially devastating attack by discovering that 1,681 API tokens were at risk of being compromised. The tokens were disc…',
 'url': 'https://biztoc.com/x/49a7e5691d3a7b0c',
 'urlToImage': 'https://c.biztoc.com/p/49a7e5691d3a7b0c/s.webp',
 'publishedAt': '2023-12-04T17:50:08Z',
 'content': 'Further validating how brittle the security of generative AI models and their platforms are, Lasso Security helped Hugging Face dodge a potentially devastating attack by discovering that 1,681 API to… [+292 chars]'}

In [7]:
all_titles = [article['title'] for article in all_articles['articles']]
all_titles[:20]

['Researchers scanned public repos and found 1,681 exposed Hugging Face API tokens belonging to Meta, Microsoft, Google, and others, many with write permissions',
 'How to choose between ChatGPT and Open Source LLMs in Finance',
 "ChatGPT was treated like the 'second coming of the messiah' and its impact was a big surprise, says Meta's AI chief",
 'Show HN: AI That Studies for You',
 "ChatGPT was treated like the 'second coming of the messiah' and its impact was a big surprise, says Meta's AI chief",
 'Ambarella : Q3 FY2024 Earnings Call Transcript',
 'Package and deploy classical ML and LLMs easily with Amazon SageMaker, part 1: PySDK Improvements',
 'Alibaba releases 72B LLM with 32k context length',
 'Revolutionizing Business Solutions with SAP BTP: A New Era of LLM Agnosticism',
 'Use Ollama LLM Models Locally with Laravel',
 'Operationalize LLM Evaluation at Scale using Amazon SageMaker Clarify and MLOps services',
 'AWS unveils new tools and services for ‘supernova’ of generative

In [8]:
articles_text = [article['content'] for article in all_articles['articles']]
articles_text[:5]

['Further validating how brittle the security of generative AI models and their platforms are, Lasso Security helped Hugging Face dodge a potentially devastating attack by discovering that 1,681 API to… [+292 chars]',
 'How to choose between ChatGPT and Open Source LLMs in FinanceMany consulting companies present LLMs and GenAI products to CEOs, CFOs, COOs, and CTOs. While these products may seem appealing, companie… [+227 chars]',
 'None Meta\'s chief scientist says the impact of ChatGPT was a surprise.None Yann LeCun told Forbes the bot was treated like the "second coming of the messiah."It\'s been a year since OpenAI launched Ch… [+281 chars]',
 '10x-nerd-buddy\r\nThis is a thing that studies for US.\r\nNERD BUDDY scours the internet for resources related to what YOU want to study\r\nWe give NERD BUDDY a study topic (a prompt) that is used to build… [+1026 chars]',
 'Yann LeCun is Meta\'s chief scientist.Meta PlatformsMeta\'s chief scientist says the impact of ChatGPT was a surprise

In [9]:
# Building the RAG based system vs a fine-tuned model
# Storing articles in a vector store

from llama_index import VectorStoreIndex, ServiceContext, Document
from openai import OpenAI
import os

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=OPENAI_API_KEY)
# Specify documents to be retrieved
documents = [Document(text=t) for t in articles_text]
# Create the vector store that we use to find relevant documents
index = VectorStoreIndex.from_documents(documents)
# A query engine is our final goal. The thing we can query
query_engine = index.as_query_engine(similarity_top_k=2)

In [14]:
# Querying the engine with new text
response = query_engine.query("How is NVidia doing in the ML space?")
response.

AttributeError: 'Response' object has no attribute 'keys'

In [15]:
# Search util to omit all the extra stuff
def search(query):
    response = query_engine.query(query)
    output = {
        'response': response.response,
        'retrieved_nodes': [p.text for p in response.source_nodes],
    }
    return output

In [16]:
search("What AWS service can I use to deploy Llama2 models?")

{'response': 'Amazon Bedrock',
 'retrieved_nodes': ['Today, were announcing the availability of Metas Llama 2 Chat 13B large language model (LLM) on Amazon Bedrock. With this launch, Amazon Bedrock becomes the first public cloud service to offer a full… [+8111 chars]',
  'Run Large-Language Models (LLMs) directly in your browser ! \r\nLearn More: API Reference\r\nDeveloped By: RDS \r\nThis web demo enables you to run LLM models from Hugging Face (GGUF/GGML/tiny-llama2/starc… [+34 chars]']}